In [1]:
import json
import pandas as pd
import numpy as np
from numpy import array

import torch
import torch.nn as nn
import torch.optim as optim


#from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon")

Mounted at /content/drive


In [3]:
#%run data_prep_10step_5horizon.ipynb
%run LSTM_Model_Classes.ipynb

In [18]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/data")
df_rf_15 = pd.read_csv('df_rf_15.csv')
df_rf_20 = pd.read_csv('df_rf_20.csv')
df_rf_25 = pd.read_csv('df_rf_25.csv')
df_rf_30 = pd.read_csv('df_rf_30.csv')
df_rf_35 = pd.read_csv('df_rf_35.csv')
df_rf_40 = pd.read_csv('df_rf_40.csv')
df_rf_45 = pd.read_csv('df_rf_45.csv')
df2_allhero = pd.read_csv('df2_allhero.csv')
df2_all_remain = pd.read_csv('df2_all_remain.csv')
heroes= pd.read_csv('heroes.csv')

In [33]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/")
%run Useful_Functions.ipynb


In [6]:
#df2_allhero

In [39]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/698/10step_5horizon/models")

model_dict = torch.load('10step_5horizon_RMSE_84.pth')
#best_model_state = torch.load('10step_5horizon_best_model_RMSE.pth')
#best_model_state = torch.load('10step_5horizon_best_model_MAE.pth')

In [ ]:
df2_70

In [40]:
lookback = 10
horizon = 5

process_embedding = ProcessEmbedding(df2_allhero, embedding_dim=84, lookback=lookback) # we create the embedding vector on unsplit data to ensure all unique hero id's are contained

input_size = process_embedding.embedding_dim + 1 #84 + 1
hidden_size = 64
num_layers = 2
output_size = 5  # Assuming you want to predict 5 values

model = LSTMModel(input_size, hidden_size, num_layers, output_size, process_embedding)

In [41]:
index = 5  # Choose the index of the desired time series
selected_row = df2_all_remain.iloc[index]

# Extract the necessary information from the selected row
hero_id = selected_row['hero_id']
time_series = np.array(json.loads(selected_row['gold_t'])).astype('float32')

# Scale the time series using the same scaling function used during training
scaled_time_series = ConstantMinMaxScaler(time_series, min_gold, max_gold)

# Create windows from the scaled time series
X, y = create_windows(scaled_time_series, lookback, horizon = 5)


# Create a batch with the selected time series
batch = ((hero_id,), (X,), (y,))

# Load our saved model state
model.load_state_dict(model_dict)

# Set the model to evaluation mode
model.eval()

# Disable gradient computation
with torch.no_grad():
    # Forward pass
    outputs = model(batch)

    # Get the predicted values
    predicted = outputs.squeeze().numpy()


#Unscaling values
unscaled_targets = ConstantUnScaler(y[:,-5:], min_gold, max_gold)
unscaled_preds = ConstantUnScaler(predicted, min_gold, max_gold)

# Print the Time Series
#print("Timeseries:")
#print(time_series)
# Print the predicted values
#print("Targets:")
#print(y[:,-5:])
#print("Predicted values:")
#print(predicted)

# Print Unscaled Targets
#print("Unscaled Targets")
#print(unscaled_targets)
#print("Unscaled Preds")
#print(unscaled_preds)

RuntimeError: expected m1 and m2 to have the same dtype, but got: double != float

In [ ]:
scaled_targets = y[:,-5:]
scaled_preds = predicted

#print(scaled_targets)
#print(scaled_preds)

In [ ]:
# print(unscaled_preds.shape)
# print(lookback+horizon-1) # 10 + 5 - 1 = 14
# print(len(unscaled_preds)+horizon-1) # 31 + 5 - 1 = 35
# print(len(preds_plot)) # 45
# print(unscaled_preds[-5:])
# #35-14 = 21
# preds_plot
# time_series

In [ ]:
# create an array from the unscaled_preds array that contains the values from the first row and the last value of each subsequent row
row_one = unscaled_preds[0,]
last_of_rest = unscaled_preds[1:,-1]
unscaled_preds_series = np.ones_like(time_series) * np.nan
unscaled_preds_series[lookback:len(time_series)] =  np.concatenate((row_one, last_of_rest), axis=0)

In [ ]:
print(unscaled_targets)

print(unscaled_preds_series)

In [ ]:
import matplotlib.pyplot as plt

horizon = 5  # Number of steps to predict

#preds_plot = np.ones_like(time_series) * np.nan
#preds_plot[lookback+horizon-1:len(unscaled_preds)+horizon-1] = unscaled_preds[-5:]
#preds_plot[14:35] =
# Plot actual values
plt.plot(time_series, label="Whole Timeseries")

# Plot predicted values
#plt.plot(unscaled_targets, c='g', label="Target Values")
plt.plot(unscaled_preds_series, c = 'r', label = "Forecasting Predictions")

plt.title(f"HeroID:{hero_id} Game Length: {len(time_series)} Minutes, Lookback Window: {lookback}")

# Add a label to the x-axis
plt.xlabel('Timestep (Minute)')

# Add a label to the y-axis
plt.ylabel('Gold')
plt.legend()
plt.show()

In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# Create an empty dataframe to store the average RMSE scores for each hero
df_results_all = pd.DataFrame(columns=['hero_id','match_id','rmse'])

# Get the unique hero_ids from the dataframe
hero_ids = df_all_remain['hero_id'].unique()

# Loop over each hero_id
for hero_id in hero_ids:
    #print("Hero_ID: ", hero_id)
    # Filter the dataframe for the current hero_id
    hero_df = df_all_remain[df_all_remain['hero_id'] == hero_id]
    #hero_df = hero_df.iloc[:3] # Used to restrict how large the df is
    #print("Hero_df: ", hero_df)

    # Create empty lists to store the unscaled targets and predictions for the current hero
    #unscaled_targets_list = []
    #unscaled_preds_list = []

    # Loop over each record for the current hero
    for index, row in hero_df.iterrows():
        # Extract the necessary information from the row
        match_id = row['match_id']
        print("Hero ID: ", hero_id, "Index: ", index, "Match ID: ", match_id)
        time_series = np.array(row['gold_t']).astype('float32')

        # Scale the time series using the same scaling function used during training
        scaled_time_series = ConstantMinMaxScaler(time_series, min_gold, max_gold)

        # Create windows from the scaled time series
        X, y = create_windows(scaled_time_series, lookback , horizon)

        # Create a batch with the selected time series
        batch = ((hero_id,), (X,), (y,))


        # Set the model to evaluation mode
        model.eval()

        # Disable gradient computation
        with torch.no_grad():
            # Forward pass
            outputs = model(batch)

            # Get the predicted values
            predicted = outputs.squeeze().numpy()
            #print("Predicted: ", predicted)

        #print("Y: ", y)
        #print("Y[:-5]: ", y[:,-5:])
        #print("Predicted: ", predicted)
        # Unscaling values
        unscaled_target = ConstantUnScaler(y[:,-5:], min_gold, max_gold)
        unscaled_pred = ConstantUnScaler(predicted, min_gold, max_gold)

        # Append the last unscaled target and prediction to the respective lists
        #unscaled_targets_list.append(unscaled_target)
        #unscaled_preds_list.append(unscaled_pred)

    # Calculate the RMSE for the current hero

        #print("Unscaled Targets: ", unscaled_target)
        #print("Unscaled Preds: ", unscaled_pred)
        rmse = np.sqrt(mean_squared_error(unscaled_target, unscaled_pred))

    # Create a dataframe for the current hero with RMSE value

        df_results_hero = pd.DataFrame({'hero_id': hero_id, 'match_id': match_id, 'rmse': rmse, 'unscaled_target': [unscaled_target], 'unscaled_pred':[unscaled_pred]})
        #print("DF Results Hero: ", df_results_hero)

    # Append the hero_id and RMSE to the df_results_all dataframe
        df_results_all = pd.concat([df_results_all, df_results_hero], ignore_index = True)

In [ ]:
df_results_all